In [129]:
# ────────────────────────────────────────────────
# 1. Cargar librerías necesarias
# ────────────────────────────────────────────────

import os
# Permite acceder a variables de entorno (como claves de API en un archivo .env)

import time
# Permite hacer pausas controladas (ej. time.sleep(1) = espera 1 segundo) para no saturar APIS.

import pandas as pd  
# Permite trabajar con tablas (como Excel, pero en código)

import spotipy  
# Librería oficial para conectarse a Spotify

from spotipy.oauth2 import SpotifyClientCredentials  
# OAuth es un protocolo de autorización seguro que permite a tu app acceder a Spotify usando credenciales de cliente (ID + secreto)

from dotenv import load_dotenv  
# → Herramienta para leer el archivo '.env' donde guardas tus claves secretas
# Así no las dejas expuestas en el código


In [56]:
pip show python-dotenv

Name: python-dotenv
Version: 1.2.1
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: 
Author: 
Author-email: Saurabh Kumar <me+github@saurabh-kumar.com>
License: 
Location: c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: 
Required-by: webdriver-manager
Note: you may need to restart the kernel to use updated packages.


In [57]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [130]:
# ════════════════════════════════════════════════════════════════════════════
# 2. CARGAR Y VALIDAR CREDENCIALES DE SPOTIFY
# ════════════════════════════════════════════════════════════════════════════

# Carga las variables de entorno desde el archivo .env
# 'override=True' → Reemplaza valores existentes, útil al actualizar credenciales
load_dotenv('.env', override=True)

# Extrae las credenciales del entorno
id_spotify = os.getenv('SPOTIFY_CLIENT_ID')        # ID público de tu aplicación
secreto_spotify = os.getenv('SPOTIFY_CLIENT_SECRET')  # Clave secreta (¡nunca la compartas!)

# Validación de seguridad: Verifica que ambas credenciales existan
if not id_spotify or not secreto_spotify:
    raise ValueError(
        "Error: Faltan credenciales en el archivo .env"
        "   Asegúrate de tener:"
        "   - SPOTIFY_CLIENT_ID=tu_client_id"
        "   - SPOTIFY_CLIENT_SECRET=tu_client_secret"
    )

# Crea el objeto de autenticación con las credenciales validadas
mis_credenciales = SpotifyClientCredentials(
    client_id=id_spotify,
    client_secret=secreto_spotify
)

# Inicializa la conexión con la API de Spotify
spotify = spotipy.Spotify(auth_manager=mis_credenciales)
print("Conexión exitosa a la API de Spotify")

# ────────────────────────────────────────────────
# En este punto ya podemos usar spotify para hacer consultas:
# - spotify.search() → Buscar canciones, artistas, álbumes
# - spotify.artist() → Información de un artista
# - spotify.track() → Detalles de una canción
# ────────────────────────────────────────────────


Conexión exitosa a la API de Spotify


In [158]:
# ────────────────────────────────────────────────
# 3. PRUEBA DE CONEXIÓN
# ────────────────────────────────────────────────
# Búsqueda de prueba: música latina del 2016, 1 resultado
prueba = spotify.search(
    q='year:2016 genre:latin',  # Consulta: año 2016 y género flamenco
    type='track',                   # Tipo: canciones
    limit= 1                       # Solo 1 resultado
)

# Crear DataFrame
cancion = prueba['tracks']['items'][0]
df_prueba = pd.DataFrame([{
    'artista': cancion['artists'][0]['name'],
    'album': cancion['album']['name']
}])

print("Conexión exitosa")
print(df_prueba)


Conexión exitosa
     artista  album
0  Bad Bunny  Diles


In [170]:
# ────────────────────────────────────────────────
# 4. BÚSQUEDA DE ARTISTAS Y ÁLBUMES POR GÉNERO Y AÑO
# ────────────────────────────────────────────────

# LÓGICA DEL CÓDIGO:
# La API de Spotify NO permite buscar álbumes directamente por género.
# Según la documentación oficial (https://developer.spotify.com/documentation/web-api/reference/search):
# - El filtro "genre:" SOLO funciona con type='artist' y type='track'
# - El filtro "year:" SÍ funciona con type='album', type='artist' y type='track'
# 
# Por eso, la estrategia es:
# 1. Buscar ARTISTAS por género (esto SÍ funciona)
# 2. Buscar ÁLBUMES de esos artistas filtrados por año (esto SÍ funciona)
# 3. Obtener las CANCIONES de esos álbumes para ese año

# Lista de géneros musicales que queremos buscar
generos = ["flamenco", "latin", "jazz", "rock"]

# El año de la música que buscamos
año = 2016

# Lista vacía para guardar todas las canciones que encontremos
canciones = []

# Mensaje inicial
print("Buscando música del año", año)
print()

# Recorremos cada género uno por uno
for genero in generos:
    
    print("Buscando género:", genero)
    
    # PASO 1: Buscar artistas de este género
    # Usamos type='artist' porque "genre:" SOLO funciona con artistas
    # Documentación: https://developer.spotify.com/documentation/web-api/reference/search
    print("  Buscando artistas de", genero)
    resultado_artistas = spotify.search(
        q='genre:' + genero,      # Filtro por género (solo válido con type='artist')
        type='artist',             # Tipo de búsqueda: artistas
        limit=10                   # Máximo 10 artistas por género
    )
    
    # Extraer la lista de artistas del resultado
    lista_artistas = resultado_artistas['artists']['items']
    
    print("  Se encontraron", len(lista_artistas), "artistas")
    
    # Lista para controlar qué álbumes ya guardamos (sin repetir)
    albumes_guardados = []
    
    # Ahora recorremos cada artista que encontramos
    for artista in lista_artistas:
        
        # Guardamos el nombre del artista
        nombre_artista = artista['name']
        
        print("    Buscando álbumes de:", nombre_artista)
        
        # PASO 2: Buscar álbumes de este artista del año que queremos
        # Usamos type='album' con los filtros "artist:" y "year:"
        # Esto SÍ funciona porque "year:" es compatible con álbumes
        resultado_albumes = spotify.search(
            q='artist:' + nombre_artista + ' year:' + str(año),  # Filtro por artista Y año
            type='album',                                         # Tipo de búsqueda: álbumes
            limit=10                                              # Máximo 10 álbumes por artista
        )
        
        # Extraer la lista de álbumes del resultado
        lista_albumes = resultado_albumes['albums']['items']
        
        # Recorremos cada álbum que encontramos
        for album in lista_albumes:
            
            # Guardamos la información del álbum
            id_album = album['id']
            nombre_album = album['name']
            
            # ¿Ya tenemos este álbum? Si sí, lo saltamos
            # Esto evita duplicados (un álbum puede aparecer varias veces)
            if id_album in albumes_guardados:
                continue
            
            # Marcamos que ya guardamos este álbum
            albumes_guardados.append(id_album)
            
            print("      Guardando álbum:", nombre_album)
            
            # PASO 3: Obtener todas las canciones de este álbum
            # Usamos album_tracks() para obtener las pistas del álbum
            # Documentación: https://developer.spotify.com/documentation/web-api/reference/get-an-albums-tracks
            resultado_pistas = spotify.album_tracks(id_album)
            lista_pistas = resultado_pistas['items']
            
            # Recorremos cada canción del álbum
            for pista in lista_pistas:
                
                # Creamos un diccionario con la información de la canción
                info_cancion = {
                    'nombre': pista['name'],              # Nombre de la canción
                    'artista': nombre_artista,            # Nombre del artista
                    'album': nombre_album,                # Nombre del álbum
                    'genero': genero,                     # Género musical (asignado por nosotros)
                    'año': año,                           # Año de lanzamiento
                    'id': pista['id']                     # ID único de Spotify
                }
                
                # Añadimos la canción a nuestra lista
                canciones.append(info_cancion)
    
    print("  Total de álbumes guardados:", len(albumes_guardados))
    print()

# Mostramos el resumen final
print("-----------------------------------")
print("Total de canciones encontradas:", len(canciones))
print("-----------------------------------")

Buscando música del año 2016

Buscando género: flamenco
  Buscando artistas de flamenco
  Se encontraron 1 artistas
    Buscando álbumes de: LaRumbé
  Total de álbumes guardados: 0

Buscando género: latin
  Buscando artistas de latin
  Se encontraron 10 artistas
    Buscando álbumes de: Bad Bunny
      Guardando álbum: Diles
      Guardando álbum: Soy Peor
      Guardando álbum: 1986
    Buscando álbumes de: ROSALÍA
      Guardando álbum: Instinto de Sobrevivência
      Guardando álbum: Cuore Di Padre
      Guardando álbum: Soy de Ti
    Buscando álbumes de: Beéle
      Guardando álbum: Alan Beeler: Vocal Music
    Buscando álbumes de: JC Reyes
    Buscando álbumes de: Anuel AA
      Guardando álbum: Soldado Y Profeta (Remix) [feat. Ozuna, Almighty, Kendo & Ñengo Flow]
      Guardando álbum: Check
      Guardando álbum: No Love
      Guardando álbum: Nacimos Pa Morir (feat. Jory Boy)
      Guardando álbum: 47
      Guardando álbum: Sola (Remix)
      Guardando álbum: En El Castillo
   